In [ ]:
import pandas as pd
import os

In [ ]:
pwd = os.getcwd()
print(pwd)

In [ ]:
#get the dataset from excel
dataset = pd.read_excel(pwd + "\\Data - Survey Monkey Output Edited.xlsx", sheet_name="Edited_Data")
dataset

In [ ]:
dataset_modified = dataset.copy()
dataset_modified

In [ ]:
dataset_modified.columns

In [ ]:
columns_to_drop= ['Start Date', 'End Date', 'Email Address','First Name', 'Last Name', 'Custom Data 1']
columns_to_drop


In [ ]:
dataset_modified = dataset_modified.drop(columns=columns_to_drop)


In [ ]:
dataset_modified.columns

In [ ]:
dataset_modified.columns[:8]
id_vars = list(dataset_modified.columns)[:8]
value_vars = list(dataset_modified.columns)[8:]

In [ ]:
dataset_melted = dataset_modified.melt(id_vars=id_vars, value_vars=value_vars, var_name="Question+Subquestion", value_name="Answer")
dataset_melted

In [ ]:
#get another dataset from excel
question_import = pd.read_excel(pwd + "\\Data - Survey Monkey Output Edited.xlsx", sheet_name="Question")
question_import

In [ ]:
question = question_import.copy()
question.drop(columns= ["Raw_Questions","Raw_Subquestions","Subquestions"], inplace= True)

In [ ]:
question

In [ ]:
dataset_merged = pd.merge(left=dataset_melted, right=question, how="left", left_on="Question+Subquestion", right_on="Question+Subquestion")
print("Original Data", len(dataset_melted))
print("Merged data", len(dataset_merged))
dataset_merged

In [ ]:
dataset_merged.groupby("Questions")["Answer"].count().reset_index()

In [ ]:
dataset_merged.groupby("Questions")["Respondent ID"].nunique().reset_index()

In [ ]:
respondants = dataset_merged[dataset_merged["Answer"].notna()]
respondants = respondants.groupby("Questions")["Respondent ID"].nunique().reset_index()
respondants.rename(columns={"Respondent ID":"Respondants"}, inplace=True)
respondants

In [ ]:
dataset_merged_two = pd.merge(left=dataset_merged, right=respondants, how="left", left_on="Questions", right_on="Questions")
print("Original Data", len(dataset_merged))
print("Merged data", len(dataset_merged_two))
dataset_merged_two

In [ ]:
same_answer = dataset_merged #[dataset_merged["Answer"].notna()]
same_answer = same_answer.groupby(["Question+Subquestion","Answer"])["Respondent ID"].nunique().reset_index()
same_answer.rename(columns={"Respondent ID":"Same Answer"}, inplace=True)
same_answer

In [85]:
dataset_merged_three = pd.merge(left=dataset_merged_two, right=same_answer, how="left", left_on=["Question+Subquestion","Answer"], right_on=["Question+Subquestion","Answer"])
dataset_merged_three["Same Answer"].fillna(0, inplace=True)
print("Original Data", len(dataset_merged_two))
print("Merged data", len(dataset_merged_three))
dataset_merged_three

Original Data 17028
Merged data 17028


,Respondent ID,Identify which division you work in.-Response,Identify which division you work in.-Other (please specify),Which of the following best describes your position level?-Response,Which generation are you apart of?-Response,Please select the gender in which you identify.-Response,Which duration range best aligns with your tenure at your company?-Response,Which of the following best describes your employment type?-Response,Question+Subquestion,Answer,Questions,Respondants,Same Answer
0,5379192392,Infrastructure,NaN,Staff,Generation X (born between 1965-1980),Male,0-2 years,Full time Employee,Question 1-Response,NaN,Question 1,119,0.0
1,2658722536,Finance,NaN,Staff,NaN,NaN,10+ years,Full time Employee,Question 1-Response,Answer 4,Question 1,119,17.0
2,4044163394,Infrastructure,NaN,Department Lead,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Question 1-Response,Answer 5,Question 1,119,22.0
3,5535865599,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Non-Binary,5-10 years,Full time Employee,Question 1-Response,Answer 1,Question 1,119,14.0
4,3356802928,Port Operations,NaN,Manager,Generation X (born between 1965-1980),Female,10+ years,Full time Employee,Question 1-Response,NaN,Question 1,119,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17023,7940065082,Infrastructure,NaN,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 30-Response 3,Answer 8,Question 30,182,14.0
17024,5157705612,Finance,NaN,Staff,Millennial (born between 1981-2000),Female,5-10 years,Full time Employee,Question 30-Response 3,Answer 6,Question 30,182,20.0
17025,9920755555,Port Operations,NaN,Staff,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30-Response 3,NaN,Question 30,182,0.0
17026,6638341389,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30-Response 3,NaN,Question 30,182,0.0


In [ ]:
output_dataset = dataset_merged_three.copy()
output_dataset.rename(columns={"Identify which division you work in.-Response":"Division Primary",	"Identify which division you work in.-Other (please specify)": "Division Secondary",
"Which of the following best describes your position level?-Response": "Position",	"Which generation are you apart of?-Response":"Generation",
"Please select the gender in which you identify.-Response": "Gender",	"Which duration range best aligns with your tenure at your company?-Response": "Tenure",
	"Which of the following best describes your employment type?-Response":"Employment Type"})

In [ ]:
{"Identify which division you work in.-Response",	"Identify which division you work in.-Other (please specify)",	"Which of the following best describes your position level?-Response",	"Which generation are you apart of?-Response",
"Please select the gender in which you identify.-Response",	"Which duration range best aligns with your tenure at your company?-Response",	"Which of the following best describes your employment type?-Response"}